In [ ]:
DATA_PATH = '/content/drive/MyDrive/text2food/data_process/processed_data_1.jsonl'

# Reading sample prompts

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

sample = True
with open(DATA_PATH, 'r') as fp:
    if sample:
        prompts = []
        for i in range(100_000):
            prompt = json.loads(fp.readline())
            prompts.append(prompt)
    else:
        prompts = [json.loads(line) for line in fp.readlines()]
        #prompts= prompts [ : 100_000]

df = pd.DataFrame(prompts)

# %%
# Check if GPU is available
import torch
from transformers import pipeline

#device = 0 if torch.cuda.is_available() else -1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device,
                      )

In [ ]:
prompts = df.matched_text.to_list()
indices = df.index
candidate_labels = ['describing food', 'not describing food', 'describing a beverage']
results = []

from tqdm import tqdm
bs = 1024
for i in tqdm(range(0, len(prompts), bs)):
    batch = [i for i in prompts[i: i + bs]]
    res = classifier(batch, candidate_labels)
    results.extend(res)
# %%
df_res = pd.DataFrame(results).drop(columns=['sequence'])
df_res.index = indices
#%%
df = df.join(df_res)

In [ ]:
df.to_csv('/content/drive/MyDrive/text2food/data_process/processed_data_1_classified_0.csv')
